In [1]:
import copy
from pathlib import Path
import warnings
import random

import numpy as np
import pandas as pd
import pytorch_lightning as pl
from pytorch_lightning.callbacks import EarlyStopping, LearningRateMonitor
from pytorch_lightning.loggers import TensorBoardLogger
import torch

import matplotlib.pyplot as plt

from pytorch_forecasting import Baseline, TemporalFusionTransformer, TimeSeriesDataSet
from pytorch_forecasting.data import GroupNormalizer
from pytorch_forecasting.metrics import SMAPE, PoissonLoss, QuantileLoss
from pytorch_forecasting.models.temporal_fusion_transformer.tuning import optimize_hyperparameters

In [2]:
input_length = 168
output_length = 24

show_graph = False

In [3]:
# model_paths = ["Save_File/experiment/A7/4108.ckpt",
#                "Save_File/experiment/A7/A7.ckpt"]

# building_num = [41,41]

model_paths = ["Save_File/experiment/E5/E5.ckpt",
               "Save_File/experiment/E5/E501.ckpt",
               "Save_File/experiment/E5/E502.ckpt",
               "Save_File/experiment/E5/E503.ckpt",
               "Save_File/experiment/E5/E504.ckpt",
               "Save_File/experiment/E5/E505.ckpt",
               "Save_File/experiment/E5/E506.ckpt",
               "Save_File/experiment/E5/E507.ckpt",
               "Save_File/experiment/E5/E508.ckpt",
               "Save_File/experiment/E5/E509.ckpt"]

building_num = [59 for i in range(12)]
# building_num = [36 for i in range(12)]
# building_num = [60 for i in range(12)]

data_path = "./Refined_Data/Grouped_Data/Input_Data3.csv"

In [4]:
dataframes = pd.read_csv(data_path, parse_dates = ["date_time"])

dataframes['num']     =   dataframes['num'].apply(str)
dataframes['Week']    =   dataframes['Week'].apply(str)
dataframes['24Hour']  =   dataframes['24Hour'].apply(str)
dataframes['holiday'] =   dataframes['holiday'].apply(str)
dataframes['Weekend'] =   dataframes['Weekend'].apply(str)
dataframes['energy_group'] = dataframes['energy_group'].apply(str)

# random_choice = random.randrange(2,13)

# start_point = -168 * random_choice - 1
# duration = 168

def smape(A, F):
    return 100/len(A) * np.sum(2 * np.abs(F - A) / (np.abs(A) + np.abs(F)))

all_smape = [[] for i in range(len(model_paths))]
step_smape = [[] for i in range(2, 13)]
step_mean_smape = []

In [5]:
for random_choice in range(2, 13):

    print(f"selected time slice : {random_choice}")
    start_point = -168 * random_choice - 1
    duration = 168

    for i in range(len(model_paths)):
        now_building_dataframe = dataframes.loc[dataframes["num"] == str(building_num[i])].copy()
        now_building_dataframe = now_building_dataframe[start_point:start_point+input_length+duration]
        
        now_best_tft = TemporalFusionTransformer.load_from_checkpoint(model_paths[i])

        original = now_building_dataframe.iloc[0:input_length]["kWH"].tolist()
        prediction = now_building_dataframe.iloc[0:input_length]["kWH"].tolist()  

        total_cycle = ((len(now_building_dataframe) - input_length - output_length) // output_length) + 1

        for k in range(total_cycle):
            now_start_point = k*output_length

            encoder_data = now_building_dataframe.iloc[now_start_point:now_start_point+input_length].copy()
            decoder_data = now_building_dataframe.iloc[now_start_point+input_length:now_start_point+input_length+output_length].copy()
            new_prediction_data = pd.concat([encoder_data, decoder_data], ignore_index=True)

            raw_predictions = now_best_tft.predict(new_prediction_data, mode="prediction").numpy().tolist()[0]
            originals = now_building_dataframe.iloc[now_start_point+input_length : now_start_point+input_length+output_length]["kWH"].tolist()

            prediction.extend(raw_predictions)
            original.extend(originals)

            # for next step, change dataframe's original value to predicted value
            for p in range(output_length):
                now_building_dataframe.iloc[now_start_point+input_length+p, now_building_dataframe.columns.get_loc("kWH")] = raw_predictions[p]
            
        if show_graph:
            plt.rcParams["figure.figsize"] = (17,5)
            fig = plt.figure()
            graph = fig.add_subplot(1, 1, 1)
            graph.plot(prediction, color='blue')
            graph.plot(original, color='red')
            plt.show()

        smape_loss = smape(np.array(prediction[168:]), np.array(original[168:]))
        print(f"building : {model_paths[i]}, score : {smape_loss}")
        all_smape[i].append(smape_loss)
        step_smape[random_choice-2].append(smape_loss)
    print(f"this slice : {random_choice}, score : {np.mean(np.array(step_smape[random_choice-2]))}")
    step_mean_smape.append(np.mean(np.array(step_smape[random_choice-2])))

selected time slice : 2
building : Save_File/experiment/E5/E5.ckpt, score : 9.089276792809121
building : Save_File/experiment/E5/E501.ckpt, score : 9.330264929399647
building : Save_File/experiment/E5/E502.ckpt, score : 9.959536200111332
building : Save_File/experiment/E5/E503.ckpt, score : 9.912263735472408
building : Save_File/experiment/E5/E504.ckpt, score : 8.084146779157209
building : Save_File/experiment/E5/E505.ckpt, score : 6.949988810577005
building : Save_File/experiment/E5/E506.ckpt, score : 9.252613797545402
building : Save_File/experiment/E5/E507.ckpt, score : 7.621718061247218
building : Save_File/experiment/E5/E508.ckpt, score : 10.645204163379011
building : Save_File/experiment/E5/E509.ckpt, score : 9.86012137123163
this slice : 2, score : 9.070513464092999
selected time slice : 3
building : Save_File/experiment/E5/E5.ckpt, score : 12.246198417829547
building : Save_File/experiment/E5/E501.ckpt, score : 10.902817169709367
building : Save_File/experiment/E5/E502.ckpt, sc

In [6]:
print(f"total score mean : {np.mean(np.array(step_mean_smape))}")
for i in range(len(all_smape)):
    print(f"index : {i}, totoal score : {np.mean(np.array(all_smape[i]))}, min : {np.min(np.array(all_smape[i]))}, max : {np.max(np.array(all_smape[i]))}")

total score mean : 11.035071004817413
index : 0, totoal score : 11.68662229455424, min : 9.089276792809121, max : 12.973770083392736
index : 1, totoal score : 11.080850930044798, min : 9.330264929399647, max : 12.449590511053302
index : 2, totoal score : 12.15002722184163, min : 9.959536200111332, max : 13.892621906721022
index : 3, totoal score : 11.196743384955356, min : 9.758531676825731, max : 12.353752703832004
index : 4, totoal score : 10.485380086931718, min : 8.084146779157209, max : 12.299413176757422
index : 5, totoal score : 9.039688185437727, min : 6.202316697972456, max : 11.453637046062878
index : 6, totoal score : 11.213608433627996, min : 9.252613797545402, max : 12.794121198727956
index : 7, totoal score : 10.158810721003944, min : 7.621718061247218, max : 11.959848955547438
index : 8, totoal score : 11.618289217869133, min : 10.225279336768775, max : 12.854491576233519
index : 9, totoal score : 11.72068957190761, min : 9.86012137123163, max : 13.475077167948204
